In [11]:
import os
from rapidocr_onnxruntime import RapidOCR
# from rapidocr_paddle import RapidOCR
engine = RapidOCR(det_use_cuda=True, cls_use_cuda=True, rec_use_cuda=True)
img_ful_path = r"E:\explore\CLI2P\dataset\001T110027850\001T110027850.jpg-0enhance-0.jpg"
engine(img_ful_path)

2024-10-10 15:33:34,677 - OrtInferSession - WARNING: CUDAExecutionProvider is not avaiable for current env, the inference part is automatically shifted to be executed under CPUExecutionProvider.
2024-10-10 15:33:34,718 - OrtInferSession - WARNING: CUDAExecutionProvider is not avaiable for current env, the inference part is automatically shifted to be executed under CPUExecutionProvider.
2024-10-10 15:33:34,858 - OrtInferSession - WARNING: CUDAExecutionProvider is not avaiable for current env, the inference part is automatically shifted to be executed under CPUExecutionProvider.


([[[[124.0, 12.0], [641.0, 5.0], [643.0, 115.0], [125.0, 123.0]],
   '老人与海',
   0.99894243],
  [[[686.0, 16.0], [772.0, 16.0], [772.0, 41.0], [686.0, 41.0]],
   'THE',
   0.99667495],
  [[[799.0, 17.0], [882.0, 17.0], [882.0, 39.0], [799.0, 39.0]],
   'OLD',
   0.9850397],
  [[[913.0, 18.0], [995.0, 18.0], [995.0, 38.0], [913.0, 38.0]],
   'MAN',
   0.59445],
  [[[685.0, 47.0], [773.0, 46.0], [773.0, 72.0], [685.0, 74.0]],
   'AND',
   0.995182],
  [[[1090.0, 40.0], [1131.0, 40.0], [1131.0, 79.0], [1090.0, 79.0]],
   '美',
   0.9997305],
  [[[1562.0, 45.0], [2091.0, 26.0], [2093.0, 78.0], [1564.0, 97.0]],
   'H319.4:1712.45/',
   0.9894315],
  [[[687.0, 76.0], [880.0, 76.0], [880.0, 101.0], [687.0, 102.0]],
   'THESEA',
   0.99528784]],
 [1.0048101000000145, 0.010423898696899414, 0.5221891403198242])

In [3]:
def get_use_content_from_ocr(reses):
    res_text = ''
    for res in reses:
        if len(res[1]) > 0:
            res_text += res[1]
            res_text += ' '
    return res_text.strip()
        

In [4]:
# img_path = r'E:\explore\CLI2P\dataset\001T110027850\001T110027850.jpg-0enhance-0.jpg'
# res, _ = engine(img_path)
# print(f'res:{res}')

In [5]:
# get_use_content_from_ocr(res)

In [6]:
dataset_path = r'./datasets'
# from tqdm import tqdm as notebook_tqdm
collect_text_len = []
collect_invaild_img_l = []

for root, dirs, files in os.walk(dataset_path):
    # print(f'root:{root}')
    # print(f'dir:{dir}')
    # print(f"file:{file}")
    for file in files:
        if file.endswith(('.jpg','.png')):
            
            img_ful_path = os.path.join(root, file)
            txt_ful_path = os.path.splitext(img_ful_path)[0] + '.txt'
            # print(img_ful_path)
            # print(txt_ful_path)
            res, _ = engine(img_ful_path)
            # print(f'res:{res}')
            if res is not None:
                res_str = get_use_content_from_ocr(res)
                # print(res_str)
                # print(f'text-len:{len(res_str)}')
                collect_text_len.append(len(res_str))
                with open(txt_ful_path, 'w', encoding='utf-8') as file:
                    file.write(res_str)
            else:
                collect_invaild_img_l.append(img_ful_path)

                    

In [7]:
len(collect_invaild_img_l)

0

In [8]:
# 删除 无效列表中的图像数据， 顺便将对应的txt标签数据也删除、
import shutil
for i in collect_invaild_img_l:
    img_ful_path = i
    txt_ful_path = os.path.splitext(img_ful_path)[0] + '.txt'
    # print('='*10)
    # print(f'img_ful_path:{img_ful_path}')
    # print(f'txt_ful_path:{txt_ful_path}')
    os.remove(img_ful_path)
    if os.path.exists(txt_ful_path):
        os.remove(txt_ful_path)

In [9]:
collect_invaild_img_l[:4]

[]

In [10]:
max(collect_text_len)

ValueError: max() arg is an empty sequence

In [11]:
sum(collect_text_len)/len(collect_text_len)

31.768123393316195

In [12]:
import statistics
median = statistics.median(collect_text_len)
print(median)


29


In [20]:
# TODO: 划分数据集
import random


def is_empty_dir(directory):
    return os.path.isdir(directory) and not os.listdir(directory)

dataset_path = r'./datasets'

prepare_dataset_path = r'./datasets_book_spine'
shutil.rmtree(prepare_dataset_path)

train_dir = os.path.join(prepare_dataset_path, 'train')
val_dir = os.path.join(prepare_dataset_path, 'val')
test_dir = os.path.join(prepare_dataset_path, 'test')
# -- 训练集 --
if not os.path.exists(train_dir):
    os.makedirs(train_dir)
# -- 验证集 --
if not os.path.exists(val_dir):
    os.makedirs(val_dir)
# -- 测试集 --
if not os.path.exists(test_dir):
    os.makedirs(test_dir)
    
for dir_name in os.listdir(dataset_path):
    if dir_name not in ['train','val','test']:
        src_dir_path = os.path.join(dataset_path, dir_name)
        
        random_number = random.random()
        print(f'random_number:{random_number}')
        if is_empty_dir(src_dir_path):
            continue
        if random_number < 0.1: # 测试集
            dst_dir_path = os.path.join(test_dir, dir_name)
        elif random_number < 0.3: # 验证集
            dst_dir_path = os.path.join(val_dir, dir_name)
        else: # 训练集
            dst_dir_path = os.path.join(train_dir, dir_name)
        shutil.copytree(src_dir_path, dst_dir_path)

random_number:0.00338631209907303
random_number:0.8096416542042781
random_number:0.15890804827201688
random_number:0.7400572819307388
random_number:0.2680067968904828
random_number:0.661637847874989
random_number:0.98854231333756
random_number:0.5843774394230927
random_number:0.8317079039009898
random_number:0.5966321988843359
random_number:0.48482172355071673
random_number:0.8707000643634275
random_number:0.5897773802527239
random_number:0.057691278949588254
random_number:0.39509214247554825
random_number:0.28925261462726504
random_number:0.8536201060091246
random_number:0.40340133279157153
random_number:0.49667158086247265
random_number:0.5677174022648342
random_number:0.4119868971367743
random_number:0.9143167929642478
random_number:0.972266495397507
random_number:0.5963007489650818
random_number:0.9770174647297855
random_number:0.12320876588537244
random_number:0.9407973837368501
random_number:0.23340675589985682
random_number:0.04517697808563281
random_number:0.2817640963564515
ra